In [ ]:
import pandas as pd

filepath = "https://github.com/data-to-insight/ERN-sessions/raw/main/data/903_xlsx_errors.xlsx"

dfs = pd.read_excel(filepath, sheet_name=None)

# dfs.keys()

# dfs['header'].info()

dfs['header'].head()

dict_keys(['ad1', 'episodes', 'header', 'missing', 'oc2', 'oc3', 'placed_for_adoption', 'previous_permenance', 'reviews', 'uasc'])

In [2]:
header = dfs['header']

# exampple - ssda903 error 102 - date of birth is not a valid date

# coercing errors returns NATs for invalid dates
# if i slice out NATs then i have all the invalid dates
header['DOB_dt'] = pd.to_datetime(header['DOB'], format="%d/%m/%Y", errors='coerce')

#.isna() returns true where the value in a row in a a column is an Na
invalid_dobs = header['DOB_dt'].isna()

#but the one above is just a condition, so here we put it into a dataframe
error_rows = header[invalid_dobs]

error_rows

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB,DOB_dt
2675,bad_entry_1,7,oops!,Alien,UPDOG,NaN,NaN,NaT


In [9]:
# 115 - Date of Local Authority's (LA) decision that a child 
# should be placed for adoption is not a valid date."

# select the placed for adoption table from dfs, 
# convert the correct column to a datetime, 
# make the slice

p4a = dfs['placed_for_adoption']

p4a['DATE_PLACED_dt'] = pd.to_datetime(p4a['DATE_PLACED'], format="%d/%m/%Y", errors='coerce')

invalid_placed_for_adoption_dates = p4a['DATE_PLACED_dt'].isna()

p4a_errors = p4a[invalid_placed_for_adoption_dates]

p4a_errors


,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED,DATE_PLACED_dt
29,bad_entry_1,Not a date,This isn't a valid date,This also isn't a date,oop,NaT


In [10]:
# .isin() -> returns true where a column has a value in a list... it is in the list
# 103 - The ethnicity code is either not valid or has not been entered.

ethncity_codes = [
        "WBRI",
        "WIRI",
        "WOTH",
        "WIRT",
        "WROM",
        "MWBC",
        "MWBA",
        "MWAS",
        "MOTH",
        "AIND",
        "APKN",
        "ABAN",
        "AOTH",
        "BCRB",
        "BAFR",
        "BOTH",
        "CHNE",
        "OOTH",
        "REFU",
        "NOBT",
    ]
# ~ the ~ reverses the logic, so here we're saying where the value is NOT in ethnicity_codes
error_rows = header[~header['ETHNIC'].isin(ethncity_codes)]
error_rows

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB,DOB_dt
2675,bad_entry_1,7,oops!,Alien,UPDOG,NaN,NaN,NaT


In [11]:
# 114 - Data entry to record the status of former carer(s) of an adopted child is invalid.
# When we do an isin we need to check the datatype of the list is the same as the values
# valid codes 0, 1
# table is ad1

ad1 = dfs['ad1']

valid_codes = [1,0] # consider the data type here. We may need to cast it to a string using astype('str') to avoid needing to search for 1,0 and "1","0".

error_rows = ad1[~ad1['FOSTER_CARE'].isin(valid_codes)]
error_rows

,CHILD,DOB,DATE_INT,DATE_MATCH,FOSTER_CARE,NB_ADOPTR,SEX_ADOPTR,LS_ADOPTR
29,bad_entry,datedate,notdate,datenot,777,Q,Will,let's go


In [19]:
# return rows where DOB after DATE_PLACED = DOB > DATE_PLACED

merged_df = p4a.merge(header[['CHILD','DOB_dt']], on='CHILD', how='left', suffixes=[None, '_head'])

error_rows = merged_df[merged_df['DOB_dt'] > merged_df['DATE_PLACED_dt']]

error_rows

,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED,DATE_PLACED_dt,DOB_dt


In [ ]:
# 302 - First episode starts before child was born.
# tables are header and episodes
# DOB from header and DECOM from episodes

episodes = dfs['episodes']

episodes['DECOM_dt'] = pd.to_datetime(episodes['DECOM'],format="%d/%m/%Y", errors='coerce')

df = episodes.merge(header[['DOB_dt', 'CHILD']], how='left')

df

#error_rows = df[df['DECOM_dt'] < df['DOB_dt']]

#error_rows

,CHILD,DECOM,RNE,LS,CIN,PLACE,PLACE_PROVIDER,DEC,REC,REASON_PLACE_CHANGE,HOME_POST,PL_POST,URN,DECOM_dt,DOB_dt
0,566910,15/02/2016,L,C2,N3,T2,PR0,14/08/2017,X1,CLOSE,NaN,NaN,4244474,2016-02-15,2008-03-27
1,566910,14/08/2017,T,C2,N3,A3,PR3,NaN,X1,NaN,EG19 7TT,WV27 9FW,8969628,2017-08-14,2008-03-27
2,384923,26/04/2016,T,L1,N8,A5,PR2,09/05/2017,X1,NaN,EH14 8EC,WD10 6ZQ,1157828,2016-04-26,2011-09-09
3,384923,09/05/2017,L,C2,N8,A5,PR2,NaN,X1,CARPL,EH14 8EC,WD10 6ZQ,1157828,2017-05-09,2011-09-09
4,709901,07/04/2016,L,V3,N8,T3,PR2,22/01/2018,X1,NaN,EE28 7VZ,WT9 9BG,5814849,2016-04-07,2013-02-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5045,152113,24/03/2018,P,J3,N7,T4,PR5,NaN,X1,CLOSE,ET29 3SV,WI26 4MP,1482729,2018-03-24,2003-06-06
5046,481370,27/02/2017,L,L2,N1,P1,PR4,16/01/2018,E14,NaN,EZ27 9FE,WK24 6XN,8556688,2017-02-27,2008-02-12
5047,791577,10/05/2016,P,V4,N1,U4,PR0,29/05/2017,E8,NaN,ED28 2LY,WF2 4RW,9873663,2016-05-10,2011-05-28
5048,238470,20/01/2017,P,J3,N5,H5,PR2,22/08/2017,X1,CARPL,EP1 7NB,WG30 3RA,8367014,2017-01-20,2013-11-21


In [ ]:
# get the first episode for each child
# sort_values()

epi_ordered = episodes.sort_values(['CHILD','DECOM_dt'], ascending=True)

first_episode = epi_ordered.drop_duplicates('CHILD', keep='first')

first_episode